In [3]:
import requests, pickle
from datetime import datetime
import json, pytz
import pandas as pd  

import schedule
import time
import os
import pytz
import datetime as dt

-------------------

In [ ]:
def download_data():
    # Here goes the code to download the data and save it to a CSV file
    # The CSV file should be saved in the current directory

    site_info = pd.read_excel(r'Solaredge site information excel file') # Site excel file path that contains site_id and site_name information
    site_id=site_info["Site ID"]
    site_name=site_info["Site Name"]

    login_url = "https://monitoring.solaredge.com/solaredge-apigw/api/login"
    panels_url = "https://monitoring.solaredge.com/solaredge-web/p/playbackData"
    SOLAREDGE_USER = "xxx" # web username
    SOLAREDGE_PASS = "xxx" # web password

    for n in range(len(site_id)):
        
        DAILY_DATA = "4"
        WEEKLY_DATA = "5"

        session = requests.session()
        try:  # Make sure the cookie file exists
            with open('solaredge.cookies', 'rb') as f:
                f.close()
        except IOError:  # Create the cookie file 
            session.post(login_url, headers = {"Content-Type": "application/x-www-form-urlencoded"}, data={"j_username": SOLAREDGE_USER, "j_password": SOLAREDGE_PASS})
            panels = session.post(panels_url, headers = {"Content-Type": "application/x-www-form-urlencoded", "X-CSRF-TOKEN": session.cookies["CSRF-TOKEN"]}, data={"fieldId": site_id[n], "timeUnit": DAILY_DATA})
            with open('solaredge.cookies', 'wb') as f:
                pickle.dump(session.cookies, f)
                f.close()
        with open('solaredge.cookies', 'rb') as f:
            session.cookies.update(pickle.load(f))
            panels = session.post(panels_url, headers = {"Content-Type": "application/x-www-form-urlencoded", "X-CSRF-TOKEN": session.cookies["CSRF-TOKEN"]}, data={"fieldId": site_id[n], "timeUnit": DAILY_DATA})
            if panels.status_code != 200:
                session.post(login_url, headers = {"Content-Type": "application/x-www-form-urlencoded"}, data={"j_username": SOLAREDGE_USER, "j_password": SOLAREDGE_PASS})
                panels = session.post(panels_url, headers = {"Content-Type": "application/x-www-form-urlencoded", "X-CSRF-TOKEN": session.cookies["CSRF-TOKEN"]}, data={"fieldId": site_id[n], "timeUnit": DAILY_DATA})
                if panels.status_code != 200:
                    exit()
                with open('solaredge.cookies', 'wb') as f:
                    pickle.dump(s.cookies, f)
            response = panels.content.decode("utf-8").replace('\'', '"').replace('Array', '').replace('key', '"key"').replace('value', '"value"')
            response = response.replace('timeUnit', '"timeUnit"').replace('fieldData', '"fieldData"').replace('reportersData', '"reportersData"')
            response = json.loads(response)
    
            data = {}
            for date_str in response["reportersData"].keys():
                date = datetime.strptime(date_str, '%a %b %d %H:%M:%S GMT %Y')
                #date = pytz.timezone('Asia/Hong_Kong').localize(date).astimezone(pytz.utc)
                for sid in response["reportersData"][date_str].keys():
                    for entry in response["reportersData"][date_str][sid]:
                        if entry["key"] not in data.keys():
                            data[entry["key"]] = {}
                        data[entry["key"]][date] = float(entry["value"].replace(",", ""))

            df = pd.DataFrame(data)
            for col in df.columns:
                if df[col].max()>900:
                    df=df.drop(col,axis=1)
            # Save the data to a CSV file in the local directory
            filename = site_name[n]+".csv"
            df.to_csv(filename, index=True)

In [ ]:
def job():
    # Get the current date in the timezone of Hong Kong SAR, China
    hk_timezone = pytz.timezone('Asia/Hong_Kong')
    today = dt.datetime.now(hk_timezone).strftime("%Y-%m-%d")

    # Create a folder with the current date as its name (if it doesn't exist)
    folder_path = r"Your local folder path to save the data\\" + today
    os.makedirs(folder_path, exist_ok=True)

    # Change the current working directory to the newly created folder
    os.chdir(folder_path)

    # Download the data and save it to a CSV file
    download_data()

In [ ]:
# Schedule the job to run every day at 11:50 pm (in the timezone of Hong Kong SAR, China)
schedule.every().day.at("23:50").do(job)

# Run the scheduled jobs
while True:
    schedule.run_pending()
    time.sleep(60)